Write code to implement a Physics-Informed Neural Network (PINN) for solving the time-independent/ time-dependent Schrödinger equation problem associated with the harmonic oscillator. Ensure that the PINN is exclusively trained using a combination of boundary and physics loss, without relying on the exact solution for training. The PINN should have the same shape as the exact solution provided for comparison during training.


  Schrödinger equation:  https://en.wikipedia.org/wiki/Schr%C3%B6dinger_equation

  example code for damped harmonic oscillator. : https://github.com/benmoseley/harmonic-oscillator-pinn-workshop/blob/main/PINN_intro_workshop.ipynb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import hermite
from scipy.special import factorial
from torch.autograd.functional import jacobian

# Constants
hbar = 1.0
m = 1.0
omega = 1.0
n = 0  # Quantum number for the ground state

# PINN Model Definition
class ComplexPINN(nn.Module):
    def __init__(self, num_inputs, num_hidden_units, num_layers):
        super(ComplexPINN, self).__init__()
        layers = [nn.Linear(num_inputs, num_hidden_units), nn.Tanh()]
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_hidden_units, num_hidden_units), nn.Tanh()]
        layers += [nn.Linear(num_hidden_units, 2)]  # Output real and imaginary parts
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        output = self.network(x)
        real, imag = output[:, 0].unsqueeze(1), output[:, 1].unsqueeze(1)
        return torch.complex(real, imag)

def schrodinger_loss(model, x_t, hbar=1.0, m=1.0, omega=1.0):
    psi = model(x_t)
    psi_real = psi.real
    psi_imag = psi.imag

    # Separate x and t for derivative computations
    x = x_t[:, 0].unsqueeze(-1)
    t = x_t[:, 1].unsqueeze(-1)

    # Enable gradient computation for x and t
    x.requires_grad_(True)
    t.requires_grad_(True)

    # Re-combine x and t after enabling gradient computation
    x_t_grad = torch.cat([x, t], dim=1)

    # Compute psi from the model again for gradient-enabled inputs
    psi = model(x_t_grad)
    psi_real = psi.real
    psi_imag = psi.imag

    # Compute gradients
    grad_psi_real_x = torch.autograd.grad(psi_real, x, torch.ones_like(psi_real), create_graph=True)[0]
    grad_psi_imag_x = torch.autograd.grad(psi_imag, x, torch.ones_like(psi_imag), create_graph=True)[0]

    grad_psi_real_t = torch.autograd.grad(psi_real, t, torch.ones_like(psi_real), create_graph=True)[0]
    grad_psi_imag_t = torch.autograd.grad(psi_imag, t, torch.ones_like(psi_imag), create_graph=True)[0]

    # Second spatial derivatives
    grad_psi_real_xx = torch.autograd.grad(grad_psi_real_x, x, torch.ones_like(grad_psi_real_x), create_graph=True)[0]
    grad_psi_imag_xx = torch.autograd.grad(grad_psi_imag_x, x, torch.ones_like(grad_psi_imag_x), create_graph=True)[0]

    # Schrödinger equation
    kinetic_real = -hbar**2 / (2 * m) * grad_psi_imag_xx
    kinetic_imag = hbar**2 / (2 * m) * grad_psi_real_xx
    potential_real = 0.5 * m * omega**2 * x.squeeze()**2 * psi_imag
    potential_imag = -0.5 * m * omega**2 * x.squeeze()**2 * psi_real

    f_real = grad_psi_imag_t - (kinetic_real + potential_real)
    f_imag = -grad_psi_real_t - (kinetic_imag + potential_imag)

    # Physics-informed loss
    loss = torch.mean(f_real**2 + f_imag**2)

    return loss



# Exact Solution for Comparison
def exact_solution(x, t, n=0):
    A_n = 1.0 / np.sqrt(2**n * factorial(n)) * (m * omega / (np.pi * hbar))**(0.25)
    phi_n = np.exp(-m * omega * x**2 / (2 * hbar))
    H_n = hermite(n)(np.sqrt(m * omega / hbar) * x)
    E_n = hbar * omega * (n + 0.5)
    psi_n_t = np.exp(-1j * E_n * t / hbar)
    return A_n * phi_n * H_n * psi_n_t

# Training Loop
def train_model(model, epochs, optimizer):
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Training domain with requires_grad=True
        x = torch.linspace(-5, 5, 100).view(-1, 1).float().requires_grad_(True)
        t = torch.linspace(0, 2 * np.pi / omega, 100).view(-1, 1).float().requires_grad_(True)
        x_t = torch.cartesian_prod(x.squeeze(), t.squeeze()).requires_grad_(True)

        loss = schrodinger_loss(model, x_t)
        loss.backward()
        optimizer.step()

        if epoch % 1000 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


model = ComplexPINN(num_inputs=2, num_hidden_units=50, num_layers=3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, epochs=10000, optimizer=optimizer)

# Visualization
x_plot = np.linspace(-5, 5, 100)
t_plot = np.array([0])  # Choose a specific time for plotting
x_t_plot = torch.tensor(np.array(np.meshgrid(x_plot, t_plot)).T.reshape(-1, 2), dtype=torch.float)

with torch.no_grad():
    psi_pred = model(x_t_plot).numpy()

psi_exact_plot = exact_solution(x_plot, t_plot, n)

plt.figure(figsize=(12, 6))
plt.plot(x_plot, psi_exact_plot.real, label='Exact Real Part', color='blue')
plt.plot(x_plot, psi_exact_plot.imag, label='Exact Imaginary Part', color='red')
plt.plot(x_plot, psi_pred.real, '--', label='PINN Real Part', color='blue')
plt.plot(x_plot, psi_pred.imag, '--', label='PINN Imaginary Part', color='red')
plt.title('Comparison of PINN and Exact Solution for Quantum Harmonic Oscillator')
plt.xlabel('Position x')
plt.ylabel('Wave Function ψ(x,0)')
plt.legend()
plt.show()

GPU is available
Epoch 0, Loss: 1.772781491279602
Epoch 1000, Loss: 4.2855357605731115e-05
